In [13]:
import nltk

# Setze den Suchpfad für NLTK-Daten
nltk.data.path.append(r'C:\Users\alexa\AppData\Roaming\nltk_data')

# Stelle sicher, dass punkt installiert ist
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', download_dir=r'C:\Users\alexa\AppData\Roaming\nltk_data')

In [14]:
import os

def load_emails_from_folder(folder_path):
    emails = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, encoding='latin-1') as f:  # <- latin-1 statt utf-8!
                emails.append(f.read())
    return emails

# Beispiel: Lade alle Spam-Mails
spam_emails = load_emails_from_folder(r'C:\Users\alexa\Documents\4. Semester Mechatronik-2025\Machine_Learning_und_Data_Science\mlds_spam_filter\data\spam')
ham_emails = load_emails_from_folder(r'C:\Users\alexa\Documents\4. Semester Mechatronik-2025\Machine_Learning_und_Data_Science\mlds_spam_filter\data\ham')


split data in train and test

In [15]:
from sklearn.model_selection import train_test_split

# 1. Labels: 1 für Spam, 0 für Ham
labels_spam = [1] * len(preprocessed_spam)
labels_ham = [0] * len(preprocessed_ham)

# 2. Alle Daten kombinieren
all_emails = preprocessed_spam + preprocessed_ham
all_labels = labels_spam + labels_ham



# 3. Aufteilen in Trainings- und Testdaten (z. B. 80 % Training, 20 % Test)
X_train_texts, X_test_texts, y_train, y_test = train_test_split(
    all_emails, all_labels, test_size=0.2, random_state=42, stratify=all_labels
)

formate the emails

In [16]:


import re
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')

# Nur beim ersten Mal nötig:
# nltk.download('punkt')

def load_emails_from_folder(folder_path):
    emails = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, encoding='latin-1') as f:
                emails.append(f.read())
    return emails

def preprocess_email(text):
    # Kleinbuchstaben
    text = text.lower()

    # URLs ersetzen
    text = re.sub(r'http\S+|www\S+|https\S+', 'URL', text)

    # Zahlen ersetzen
    text = re.sub(r'\d+', 'NUMBER', text)

    # Satzzeichen entfernen
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenisierung (für Stemming nötig)
    words = word_tokenize(text)

    # Stemming
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in words]

    return ' '.join(stemmed_words)

# === Beispielverwendung ===
spam_emails = load_emails_from_folder(r'C:\Users\alexa\Documents\4. Semester Mechatronik-2025\Machine_Learning_und_Data_Science\mlds_spam_filter\data\spam')
ham_emails = load_emails_from_folder(r'C:\Users\alexa\Documents\4. Semester Mechatronik-2025\Machine_Learning_und_Data_Science\mlds_spam_filter\data\ham')

# Vorverarbeitung anwenden
preprocessed_spam = [preprocess_email(email) for email in spam_emails]
preprocessed_ham = [preprocess_email(email) for email in ham_emails]

# Optional: Beispiel ausgeben
print("Original:\n", spam_emails[0][:500])
print("\nVorverarbeitet:\n", preprocessed_spam[0][:500])

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Original:
 mv 1 00001.bfc8d64d12b325ff385cca8d07b84288
mv 10 00010.7f5fb525755c45eb78efc18d7c9ea5aa
mv 100 00100.c60d1c697136b07c947fa180ba3e0441
mv 101 00101.2dfd7ee79ae439b8d9c38e783a137efa
mv 102 00102.2e3969075728dde7a328e05d19b35976
mv 103 00103.8c39bfed2079f865e9dfb75f4416a468
mv 104 00104.886f4a22362f4d3528c3e675878f17f7
mv 105 00105.9790e1c57fcbf7885b7cd1719fb4681b
mv 106 00106.fa6df8609cebb6f0f37aec3f70aa5b9a
mv 107 00107.f1d4194b57840ea6587b9a73ed88e075
mv 108 00108.4506c2ef846b80b9a7beb90315b227

Vorverarbeitet:
 mv number numberbfcnumberdnumberdnumberbnumberffnumberccanumberdnumberbnumb mv number numbernumberfnumberfbnumbercnumberebnumberefcnumberdnumbercnumbereanumberaa mv number numbercnumberdnumbercnumberbnumbercnumberfanumberbanumberenumb mv number numbernumberdfdnumbereenumberaenumberbnumberdnumbercnumberenumberanumberefa mv number numbernumberenumberddenumberanumberenumberdnumberbnumb mv number numbernumbercnumberbfednumberfnumberenumberdfbnumberfnumberanumb mv number

In [17]:
from collections import Counter

# 1. Tokenisieren (die E-Mails wurden bereits vorverarbeitet → einfach splitten)
tokenized_spam = [email.split() for email in preprocessed_spam]
tokenized_ham = [email.split() for email in preprocessed_ham]
all_tokenized = tokenized_spam + tokenized_ham

# 2. Vokabular erstellen (z. B. Top 1000 häufigste Wörter)
def build_vocabulary(tokenized_emails, vocab_size=None):
    all_tokens = []
    for tokens in tokenized_emails:
        all_tokens.extend(tokens)
    word_counts = Counter(all_tokens)
    most_common = word_counts.most_common(vocab_size)
    vocabulary = [word for word, _ in most_common]
    return vocabulary

vocabulary = build_vocabulary(all_tokenized, vocab_size=1000)

# 3. Funktion zur Vektorerstellung
def email_to_vector(tokens, vocabulary, binary=True):
    token_counts = Counter(tokens)
    vector = []
    for word in vocabulary:
        if binary:
            vector.append(1 if word in token_counts else 0)
        else:
            vector.append(token_counts[word])
    return vector

# 4. Vektorisieren aller E-Mails
binary_vectors_spam = [email_to_vector(tokens, vocabulary, binary=True) for tokens in tokenized_spam]
binary_vectors_ham = [email_to_vector(tokens, vocabulary, binary=True) for tokens in tokenized_ham]

count_vectors_spam = [email_to_vector(tokens, vocabulary, binary=False) for tokens in tokenized_spam]
count_vectors_ham = [email_to_vector(tokens, vocabulary, binary=False) for tokens in tokenized_ham]

# 5. Beispielausgabe
print("Vokabular (Top 20):", vocabulary[:20])
print("Binär-Vektor (erste Spam-Mail):", binary_vectors_spam[0][:20])
print("Zähl-Vektor (erste Spam-Mail):", count_vectors_spam[0][:20])

Vokabular (Top 20): ['number', 'the', 'numbernumbernumb', 'from', 'to', 'for', 'with', 'by', 'receiv', 'numbernumbernumbernumb', 'of', 'and', 'a', 'id', 'sep', 'url', 'in', 'esmtp', 'localhost', 'numbernumb']
Binär-Vektor (erste Spam-Mail): [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Zähl-Vektor (erste Spam-Mail): [500, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Tokenisiere die Trainings- und Testdaten
tokenized_train = [email.split() for email in X_train_texts]
tokenized_test = [email.split() for email in X_test_texts]

# Wandle sie in Vektoren um (nutze das gleiche Vokabular wie oben)
X_train_vectors = [email_to_vector(tokens, vocabulary, binary=True) for tokens in tokenized_train]
X_test_vectors = [email_to_vector(tokens, vocabulary, binary=True) for tokens in tokenized_test]

# Klassifikatoren
classifiers = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(C=1, max_iter=1000),
    "Linear SVM": LinearSVC(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

# Alle trainieren und evaluieren
for name, clf in classifiers.items():
    clf.fit(X_train_vectors, y_train)
    y_pred = clf.predict(X_test_vectors)
    
    print(f"\n📌 {name}:\n")
    print(classification_report(y_test, y_pred, target_names=["Ham", "Spam"]))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))